In [ ]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.2 MB/s eta 0:00:00


In [ ]:
#TRAININGGGGGGGG
def read_text_file(file_path):
  with open(file_path, "r", encoding="utf-8") as file:
    content = file.read()
  return content
file_path = "/content/testtext.txt"
text = read_text_file(file_path)
print(text[:500])

hello this is clara!!!



In [ ]:
#TRAININGGGGG
def write_text_file(file_path, content):
  with open(file_path, "w", encoding= "utf-8") as file:
    file.write(content)

output_file = "/content/testtextprocessed.txt"
summary_content = "written by clara youssef"
write_text_file(output_file, summary_content)


In [ ]:
#TRAININGGGG
def append_to_text_file(file_path, additional_content):
  with open(file_path, "a", encoding = "utf-8") as file:
    file.write(additional_content)

append_to_text_file(output_file, "\nAdditional summary details.")

In [ ]:
#TRAINING COMBINED thats how we move the text from a file to a new file
def process_text_file(input_file, output_file):
  text = read_text_file(input_file)
  processed_text = text.lower().strip()
  write_text_file(output_file, processed_text)

input_file = "/content/testtextprocessed.txt"
output_file = "/content/clarafile.txt"
process_text_file(input_file, output_file)

In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests


In [ ]:
def fetch_webpage(url):
  response = requests.get(url)
  if response.status_code == 200:
    return response.text
  else:
    raise Exception(f"failed to fetch webpage. status code: {response.status_code}")
#example below
url = "https://arxiv.org/abs/2409.05400"
html_content = fetch_webpage(url)
print(html_content[:500])


<!DOCTYPE html>
<html lang="en">

<head>  <title>[2409.05400] On cat-human interaction from the viewpoint of physics: an equation of motion</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="apple-touch-icon" sizes="180x180" href="/static/browse/0.3.4/images/icons/apple-touch-icon.png">
  <link rel="icon" type="image/png" sizes="32x32" href="/static/browse/0.3.4/images/icons/favicon-32x32.png">
  <link rel="icon" type="image/png" sizes="16x16" href="/stat


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def extract_info_from_html(html_content):
  soup = BeautifulSoup(html_content, "html.parser")

  #extract metadata
  title = soup.title.string if soup.title else "No title found!"
  body_text = " ".join([p.text for p in soup.find_all("p")]) #combines all paragraphs

  return {
      "title": title.strip(),
      "body_text": body_text.strip()
  }
webpage_data = extract_info_from_html(html_content)
print("Title:", webpage_data["title"])
print("Body text(first 500 chars only):", webpage_data["body_text"][:500])

Title: [2409.05400] On cat-human interaction from the viewpoint of physics: an equation of motion
Body text(first 500 chars only): In just 3 minutes help us improve arXiv: Help | Advanced Search arXivLabs is a framework that allows collaborators to develop and share new arXiv features directly on our website. Both individuals and organizations that work with arXivLabs have embraced and accepted our values of openness, community, excellence, and user data privacy. arXiv is committed to these values and only works with partners that adhere to them. Have an idea for a project that will add value for arXiv's community? Learn mo


In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
#extract metadata from PDF
from PyPDF2 import PdfReader

def extract_pdf_metadata(pdf_path):
  try:
    reader = PdfReader(pdf_path)
    #extracting metadata
    metadata = reader.metadata
    if metadata:
      return {key[1:]: value for key, value in metadata.items()} #clean up keys
    else:
      return "No metadata found in this PDF"
  except Exception as e:
    return f"An error occurred: {e}"

pdf_path = "/content/cats_paper.pdf"
metadata = extract_pdf_metadata(pdf_path)
print("PDF Metadata:", metadata)

PDF Metadata: {'Author': '', 'CreationDate': 'D:20241106014229Z', 'Creator': 'LaTeX with hyperref', 'Keywords': '', 'ModDate': 'D:20241106014229Z', 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'Producer': 'pdfTeX-1.40.25', 'Subject': '', 'Title': '', 'Trapped': '/False'}


In [ ]:
#extract metadata from web
def extract_web_metadata(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Try to extract title from <title> tag (standard)
        title = soup.title.string if soup.title else "Title not found"

        # Try to extract author from <meta> tag (author in meta or og:author)
        author = None

        # Check for author meta tag (standard)
        author_tag = soup.find("meta", attrs={"name": "author"})
        if author_tag:
            author = author_tag.get("content", "Author not found")

        # Fallback: Try extracting from <h1> or similar elements
        if not author:
            author_tag = soup.find("h1")
            if author_tag:
                author = author_tag.string.strip() if author_tag.string else "Author not found"

        # Further fallback check for author in <meta> property og:author
        if not author:
            author_tag = soup.find("meta", attrs={"property": "og:author"})
            if author_tag:
                author = author_tag.get("content", "Author not found")

        # New fallback: Extract authors from div or span tags specific to arXiv structure
        if not author:
            author_tag = soup.find("div", class_="authors")
            if author_tag:
                author = author_tag.get_text().strip()

        # If no author found, look for a "span" with the "author" class
        if not author:
            author_tag = soup.find("span", class_="author")
            if author_tag:
                author = author_tag.get_text().strip()

        return {"Title": title, "Author": author}

    else:
        print("Failed to retrieve the webpage")
        return None

# Test the function with the URL
url = "https://arxiv.org/abs/2409.05400"  # Example URL
web_meta = extract_web_metadata(url)

if web_meta:
    print("Extracted Metadata:")
    print(f"Title: {web_meta['Title']}")
    print(f"Author: {web_meta['Author']}")
else:
    print("No metadata was extracted!")


Extracted Metadata:
Title: [2409.05400] On cat-human interaction from the viewpoint of physics: an equation of motion
Author: Physics > Physics Education


In [ ]:
#extracts detailed metadata from pdf and saves to json file
import json
from PyPDF2 import PdfReader

def extract_pdf_metadata(pdf_path, output_json_path):
    try:
        reader = PdfReader(pdf_path)
        # Extracting metadata
        metadata = reader.metadata

        if metadata:
            # Clean up and structure metadata in a well-defined format
            structured_metadata = {
                "Title": metadata.get('/Title', 'Title not found'),
                "Author": metadata.get('/Author', 'Author not found'),
                "Creator": metadata.get('/Creator', 'Creator not found'),
                "Producer": metadata.get('/Producer', 'Producer not found'),
                "CreationDate": metadata.get('/CreationDate', 'Creation date not found'),
                "ModificationDate": metadata.get('/ModDate', 'Modification date not found'),
                "Keywords": metadata.get('/Keywords', 'Keywords not found'),
                "Subject": metadata.get('/Subject', 'Subject not found'),
                "Trapped": metadata.get('/Trapped', 'Trapped not found')
            }

            # Save the structured metadata to a JSON file
            with open(output_json_path, 'w') as json_file:
                json.dump(structured_metadata, json_file, indent=4)

            return f"Metadata successfully saved to {output_json_path}"
        else:
            return "No metadata found in this PDF"
    except Exception as e:
        return f"An error occurred: {e}"

pdf_path = "/content/cats_paper.pdf"
output_json_path = "/content/pdf_metadata.json"
response = extract_pdf_metadata(pdf_path, output_json_path)
print(response)


Metadata successfully saved to /content/pdf_metadata.json


In [ ]:
#Extracting web metadata and saving it in a JSON
import requests
from bs4 import BeautifulSoup
import json

def extract_web_metadata(url, output_json_path):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract Title from <title> tag
        title = soup.title.string if soup.title else "Title not found"

        # Extract Author from <meta> tags (commonly used meta tags)
        author = None
        author_tag = soup.find("meta", attrs={"name": "author"})
        if author_tag:
            author = author_tag.get("content", "Author not found")

        # Fallback: Try extracting from <h1> or similar elements
        if not author:
            author_tag = soup.find("h1")
            if author_tag:
                author = author_tag.string.strip() if author_tag.string else "Author not found"

        # Extract additional metadata (CreationDate, ModificationDate, Keywords, Subject)
        creation_date = soup.find("meta", attrs={"name": "creation_date"})
        modification_date = soup.find("meta", attrs={"name": "modification_date"})
        keywords = soup.find("meta", attrs={"name": "keywords"})
        subject = soup.find("meta", attrs={"name": "subject"})

        # Structure metadata into a dictionary
        structured_metadata = {
            "Title": title,
            "Author": author if author else "Author not found",
            "CreationDate": creation_date.get("content", "Creation date not found") if creation_date else "Creation date not found",
            "ModificationDate": modification_date.get("content", "Modification date not found") if modification_date else "Modification date not found",
            "Keywords": keywords.get("content", "Keywords not found") if keywords else "Keywords not found",
            "Subject": subject.get("content", "Subject not found") if subject else "Subject not found"
        }

        # Save structured metadata to a JSON file
        with open(output_json_path, 'w') as json_file:
            json.dump(structured_metadata, json_file, indent=4)

        return f"Metadata successfully saved to {output_json_path}"

    else:
        return f"Failed to retrieve the webpage. Status code: {response.status_code}"

# Test the function with the URL
url = "https://arxiv.org/abs/2409.05400"  # Example URL
output_json_path = "/content/web_metadata.json"
response = extract_web_metadata(url, output_json_path)
print(response)


Metadata successfully saved to /content/web_metadata.json


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=4ade3789bff33d82b36768e0c35fe31a9e28fb5715a7d6aa52fc6ddefcd0bc2f
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

In [ ]:
!pip install pycld2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycld2: filename=pycld2-0.41-cp311-cp311-linux_x86_64.whl size=9916391 sha256=aa37bc4f71be780e10a9ec43c501aa5cabf8c1243e44df6446b4bdd1b80d4879
  Stored in directory: /root/.cache/pip/wheels/9a/71/a1/89cdc6ae2ecd7bb3bf67baee27a134f761b3e2922ea7fdfbcb
Successfully built pycld2


In [ ]:
from bs4 import BeautifulSoup
import requests
from langdetect import detect

def extract_web_metadata_with_language(url, output_json_path):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract Title from <title> tag
        title = soup.title.string if soup.title else "Title not found"

        # Extract Author from <meta> tags (commonly used meta tags)
        author = None
        author_tag = soup.find("meta", attrs={"name": "author"})
        if author_tag:
            author = author_tag.get("content", "Author not found")

        # Fallback: Try extracting from <h1> or similar elements
        if not author:
            author_tag = soup.find("h1")
            if author_tag:
                author = author_tag.string.strip() if author_tag.string else "Author not found"

        # Extract additional metadata (CreationDate, ModificationDate, Keywords, Subject)
        creation_date = soup.find("meta", attrs={"name": "creation_date"})
        modification_date = soup.find("meta", attrs={"name": "modification_date"})
        keywords = soup.find("meta", attrs={"name": "keywords"})
        subject = soup.find("meta", attrs={"name": "subject"})

        # Detect language from title and body_text
        # Handle the case where <html> might not have a "lang" attribute
        html_lang = soup.find("html")
        detected_language = None

        if html_lang:
            detected_language = html_lang.get("lang", "unknown")

        if not detected_language:
            body_text = " ".join([p.text for p in soup.find_all("p")])  # Combine all paragraphs
            try:
                detected_language = detect(body_text)
            except:
                detected_language = "unknown"

        # Structure metadata into a dictionary
        structured_metadata = {
            "Title": title,
            "Author": author if author else "Author not found",
            "CreationDate": creation_date.get("content", "Creation date not found") if creation_date else "Creation date not found",
            "ModificationDate": modification_date.get("content", "Modification date not found") if modification_date else "Modification date not found",
            "Keywords": keywords.get("content", "Keywords not found") if keywords else "Keywords not found",
            "Subject": subject.get("content", "Subject not found") if subject else "Subject not found",
            "DetectedLanguage": detected_language
        }

        # Save structured metadata to a JSON file
        with open(output_json_path, 'w') as json_file:
            json.dump(structured_metadata, json_file, indent=4)

        return f"Metadata successfully saved to {output_json_path}"

    else:
        return f"Failed to retrieve the webpage. Status code: {response.status_code}"

# Example usage
url = "https://arxiv.org/abs/2404.04167"  # Example URL
output_json_path = "/content/web_metadata_with_language.json"
response = extract_web_metadata_with_language(url, output_json_path)
print(response)


Metadata successfully saved to /content/web_metadata_with_language.json


In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [ ]:
from transformers import pipeline, MBartForConditionalGeneration, MBart50TokenizerFast


In [ ]:
import requests

# Fetch Paper Content from a URL
def fetch_paper_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text  # Or you can parse HTML if needed
    else:
        return None


In [ ]:
import requests
from transformers import pipeline
import math

# Summarize Content using Hugging Face Models in Chunks
def summarize_text_in_chunks(text, model_name="sshleifer/distilbart-cnn-12-6", max_chunk_size=256):  # Reduced chunk size
    # Initialize the summarizer
    summarizer = pipeline("summarization", model=model_name)

    # Split text into smaller chunks to avoid token limit
    num_chunks = math.ceil(len(text) / max_chunk_size)
    summaries = []

    for i in range(num_chunks):
        chunk = text[i * max_chunk_size : (i + 1) * max_chunk_size]
        print(f"Processing chunk {i + 1}/{num_chunks}")  # Track progress
        summary = summarizer(chunk, max_length=60, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    # Join all summaries to form the final summary
    return " ".join(summaries)

# Add your paper link here
paper_link = "https://hal.science/hal-01941315v1"

# Fetch the content of the paper from the URL
response = requests.get(paper_link)

# Assuming the paper is in text format and its content is in the response text
if response.status_code == 200:
    paper_content = response.text
    print("Paper content fetched successfully.")

    # Get the summary of the paper (now in chunks)
    summary = summarize_text_in_chunks(paper_content, model_name="facebook/bart-large-cnn")

    print("Summary of the paper:")
    print(summary)
else:
    print("Failed to fetch paper content.")


Paper content fetched successfully.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

...

Processing chunk 20/287
Processing chunk 21/287
Processing chunk 22/287
Processing chunk 23/287
Processing chunk 24/287
Processing chunk 25/287
Processing chunk 26/287
Processing chunk 27/287
Processing chunk 28/287
Processing chunk 29/287
Processing chunk 30/287
Processing chunk 31/287
Processing chunk 32/287
Processing chunk 33/287
Processing chunk 34/287
Processing chunk 35/287
Processing chunk 36/287
Processing chunk 37/287
Processing chunk 38/287
Processing chunk 39/287
Processing chunk 40/287
Processing chunk 41/287
Processing chunk 42/287
Processing chunk 43/287
Processing chunk 44/287
Processing chunk 45/287
Processing chunk 46/287
Processing chunk 47/287
Processing chunk 48/287
Processing chunk 49/287
Processing chunk 50/287
Processing chunk 51/287
Processing chunk 52/287
Processing chunk 53/287
Processing chunk 54/287
Processing chunk 55/287
Processing chunk 56/287
Processing chunk 57/287
Processing chunk 58/287
Processing chunk 59/287
Processing chunk 60/287
Processing chunk

Your max_length is set to 60, but your input_length is only 53. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


Processing chunk 217/287


Your max_length is set to 60, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


Processing chunk 218/287
Processing chunk 219/287
Processing chunk 220/287
Processing chunk 221/287
Processing chunk 222/287
Processing chunk 223/287
Processing chunk 224/287
Processing chunk 225/287
Processing chunk 226/287
Processing chunk 227/287
Processing chunk 228/287
Processing chunk 229/287
Processing chunk 230/287
Processing chunk 231/287
Processing chunk 232/287
Processing chunk 233/287
Processing chunk 234/287
Processing chunk 235/287
Processing chunk 236/287
Processing chunk 237/287
Processing chunk 238/287
Processing chunk 239/287
Processing chunk 240/287
Processing chunk 241/287
Processing chunk 242/287
Processing chunk 243/287
Processing chunk 244/287
Processing chunk 245/287
Processing chunk 246/287
Processing chunk 247/287
Processing chunk 248/287
Processing chunk 249/287
Processing chunk 250/287
Processing chunk 251/287
Processing chunk 252/287
Processing chunk 253/287
Processing chunk 254/287
Processing chunk 255/287
Processing chunk 256/287
Processing chunk 257/287


In [1]:
import re
def clean_text(text):
  #removing references
  text = re.sub(r'\[d+\]|(\d+\)', '', text)
  #removing special characters except for standard punctuation
  text = re.sub(r'[^a-zA-Z0-9.,!? \n]+', '', text)
  #normalize multiple spaces and line breaks to a single space
  text = re.sub(r'\s+', ' ', text).strip()
  #standardize line breaks
  text = text.replace('.', '.\n')
  return text

cleaned_summaries = [clean_text(chunk) for chunk in summarized_chunks]

for i, summary in enumerate(cleaned_summaries, 1):
  print(f"Chunk {i}:\n{summary}\n")

NameError: name 'summarized_chunks' is not defined